In [9]:
# import libraries

import numpy as np
import pandas as pd
import requests
from datetime import timedelta, datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
from sklearn.model_selection import train_test_split
from scipy import stats

# import functions

from acquire import read_csv
from prepare import missing_values_table

In [2]:
df = read_csv()

In [3]:
df.head(2)

,Month,State,Genus Species,Serotype or Genotype,Etiology Status,Location of Preparation,Illnesses,Hospitalizations,Deaths,Food Vehicle,Contaminated Ingredient
Year,,,,,,,,,,,
2009,1,Minnesota,Norovirus,NaN,Suspected,Restaurant - Sit-down dining,2,0.0,0.0,NaN,NaN
2009,1,Minnesota,Norovirus,NaN,Confirmed,NaN,16,0.0,0.0,NaN,NaN


In [5]:
df.shape

(19119, 11)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19119 entries, 2009 to 2015
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Month                    19119 non-null  int64  
 1   State                    19119 non-null  object 
 2   Genus Species            12500 non-null  object 
 3   Serotype or Genotype     3907 non-null   object 
 4   Etiology Status          12500 non-null  object 
 5   Location of Preparation  17984 non-null  object 
 6   Illnesses                19119 non-null  int64  
 7   Hospitalizations         15494 non-null  float64
 8   Deaths                   15518 non-null  float64
 9   Food Vehicle             10156 non-null  object 
 10  Contaminated Ingredient  1876 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.8+ MB


In [7]:
df.describe()

,Month,Illnesses,Hospitalizations,Deaths
count,19119.000000,19119.000000,15494.000000,15518.000000
mean,6.433443,19.537162,0.947528,0.021717
std,3.432970,49.452417,5.312772,0.386976
min,1.000000,2.000000,0.000000,0.000000
25%,4.000000,3.000000,0.000000,0.000000
50%,6.000000,8.000000,0.000000,0.000000
75%,9.000000,19.000000,1.000000,0.000000
max,12.000000,1939.000000,308.000000,33.000000


In [8]:
df.Deaths.value_counts()

0.0     15342
1.0       125
2.0        27
3.0         9
7.0         3
5.0         3
4.0         3
6.0         2
21.0        1
33.0        1
9.0         1
8.0         1
Name: Deaths, dtype: int64

In [11]:
missing_values_table(df)

Your selected dataframe has 11 columns and 19119 Rows.
There are 8 columns that have NULL values.


,Zero Values,null_count,% of Total Values,Total Zeroes + Null Values,% Total Zero + Null Values,Data Type
Contaminated Ingredient,0,17243,90.2,17243,90.2,object
Serotype or Genotype,0,15212,79.6,15212,79.6,object
Food Vehicle,0,8963,46.9,8963,46.9,object
Genus Species,0,6619,34.6,6619,34.6,object
Etiology Status,0,6619,34.6,6619,34.6,object
Hospitalizations,11439,3625,19.0,15064,78.8,float64
Deaths,15342,3601,18.8,18943,99.1,float64
Location of Preparation,0,1135,5.9,1135,5.9,object
Month,0,0,0.0,0,0.0,int64
State,0,0,0.0,0,0.0,object
